In [1]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
#from adjustText import adjust_text
import warnings
warnings.filterwarnings('ignore')

In [2]:
#print(plt.style.available)
sns.set(style="ticks", palette="bright")

In [3]:
#Reads mgf files and outputs a dataframe where MS2 events are rows. Columns are intensities of diagnostic HexNAc fragment ions
#and MS2 ID information (MS2 event precursor mass, retention time, charge, and mgf scan #)


# df_import = mgf file of protein prospector generated MS2 events

def SpecNAc(df_import):
    
    Glyc_Ions = (366.140, 204.087, 186.076, 168.066, 144.065, 138.055, 126.055, 274.092)
    cols = Glyc_Ions
    
    # Add Scan Number to Peaklist Dataframe
    df_import['scan_#'] = 0

    x = 1

    for i in range(len(df_import)):

        if (df_import.iloc[i, 0] == 'BEGIN IONS'):

            x = x

        elif (df_import.iloc[i,0] == 'END IONS'):
            x = x + 1

        df_import.at[i,'scan_#'] = x 

   
    # Clean up Data Frame by Seperating Peaks and Intensities
    df_test = df_import
    df_test['Peak'] = df_test['BEGIN IONS'].str.split(' ').str[0]
    df_test['Intensity'] = df_test['BEGIN IONS'].str.split(' ').str[1]

    
    # Converty Dataframe Entries to Numeric
    df_test.Peak = df_test.Peak.apply(pd.to_numeric, errors = 'coerce')
    df_test.Intensity = df_test.Intensity.apply(pd.to_numeric, errors = 'coerce')
    
   
    #Retrieve # of MS2 events (Scans) for use in df_final matrix
    unique_scans = df_test['scan_#'].unique()
    scans = [x for x in range(len(unique_scans))]
    
    
    #Builds Dataframe where Diagnostic Ions are Columns and Rows are Scans

    df_glyc = df_test
    df_final = pd.DataFrame(index = scans, columns = cols)
    df_final['scan_#'] = unique_scans
    df_final['RT'] = ''
    df_final['Precursor_Mass'] = ''
    df_final['Charge'] = ''

    c = 0
    i = 0

    #set tol to PPM value by formula (PPM/1,000,000)*100))
    tol = .005
   

    while i < len(df_glyc):

            #add ID columns here
            
            if 'TITLE' in df_glyc.iloc[i,0]:
                
                df_final.iloc[c,9] = df_glyc.iloc[i,0].split(' ')[2]
                df_final.iloc[c,10] = df_glyc.iloc[i+1,0].split(' ')[0]
                df_final.iloc[c,11] = df_glyc.iloc[i+2,0]
            
                i = i + 3
                        
            #add glyco peak columns
            
            elif (df_glyc.iloc[i,1] == df_final.iloc[c,8]):


                if ( abs(df_glyc.iloc[i,2] - 366.140) <= tol ):

                    df_final.iloc[c,0] = df_glyc.iloc[i,3]


                elif ( abs(df_glyc.iloc[i,2] - 204.087) <= tol ):

                    df_final.iloc[c,1] = df_glyc.iloc[i,3]


                elif ( abs(df_glyc.iloc[i,2] - 186.076) <= tol ):

                     df_final.iloc[c,2] = df_glyc.iloc[i,3] 


                elif ( abs(df_glyc.iloc[i,2] - 168.066) <= tol ):

                    df_final.iloc[c,3] = df_glyc.iloc[i,3]


                elif ( abs(df_glyc.iloc[i,2] - 144.065) <= tol ):

                    df_final.iloc[c,4] = df_glyc.iloc[i,3]


                elif ( abs(df_glyc.iloc[i,2] - 138.055) <= tol ):

                     df_final.iloc[c,5] = df_glyc.iloc[i,3]


                elif ( abs(df_glyc.iloc[i,2] - 126.055) <= tol ):

                     df_final.iloc[c,6] = df_glyc.iloc[i,3]
                
                elif ( abs(df_glyc.iloc[i,2] - 274.092) <= tol ):

                     df_final.iloc[c,7] = df_glyc.iloc[i,3]
                        

                i = i + 1

            
            #moves counter to the next scan in final dataframe 
            elif df_glyc.iloc[i,1] != df_final.iloc[c, 8]:

                c = c + 1

    
    
    df_figures = df_final
    
    df_figures['GlcRatio_AllIons'] = ( df_figures[138.055] + df_figures[168.066] ) / ( df_figures[126.055] + df_figures[144.065] )
    df_figures['GlcRatio_138_126'] = ( df_figures[138.055]  ) / ( df_figures[126.055] )
    df_figures['GlcRatio_168_144'] = ( df_figures[168.066] ) / ( df_figures[144.065] )
    df_figures['GlcRatio_138_144'] = ( df_figures[138.055] ) / (df_figures[144.065] )
    df_figures['GlcRatio_168_126'] = ( df_figures[168.066] ) / ( df_figures[126.055] )
    df_figures['GlcRatio_168_204'] = ( df_figures[168.066] ) / ( df_figures[204.087] )
    df_figures['GlcRatio_138_204'] = ( df_figures[138.055] ) / ( df_figures[204.087] )
    df_figures['GlcRatio_204_366'] = ( df_figures[204.087] ) / ( df_figures[366.140] )
    df_figures['GlcRatio_138_366'] = ( df_figures[138.055] ) / ( df_figures[366.140] )
    df_figures['GlcRatio_168_366'] = ( df_figures[168.066] ) / ( df_figures[366.140] )
    df_figures['GlcRatio_144_204'] = ( df_figures[144.065] ) / ( df_figures[204.087] )
    
    #Sialic Acid Ratios
    df_figures['GlcRatio_274_204'] = ( df_figures[274.092] ) / ( df_figures[204.087] )
    df_figures['GlcRatio_274_144'] = ( df_figures[274.092] ) / ( df_figures[144.065] )
    df_figures['GlcRatio_274_126'] = ( df_figures[274.092] ) / ( df_figures[126.055] )
    df_figures['GlcRatio_274_138'] = ( df_figures[274.092] ) / ( df_figures[138.055] )


    
    return df_figures

In [4]:
#Matches a PSM in a Byos report with an mgf MS2 event. 
#Generates a new dataframe with information from matched MS2 events for each PSM


# byos_input = Byos PSM report
# mgf_input = output of SpecNAc function

def MgfMatch(byos_input, mgf_input):


    test = mgf_input

    test['Charge'] = test['Charge'].str.replace(r'[^\d.]', '')
    test['Precursor_Mass'] = test['Precursor_Mass'].str.replace(r'[^\d.]', '')
    test['RT'] = test['RT'].str.replace(r'[^\d.]', '')

    test = test[:-1]

    test['RT'] = pd.to_numeric(test['RT'])
    test['Precursor_Mass'] = pd.to_numeric(test['Precursor_Mass'])
    test['Charge'] = pd.to_numeric(test['Charge'])



    df_new = byos_input


    df_new['GlcRatio_138_144'] = ''
    df_new['mgf_PM'] = ''
    df_new['mgf_RT'] = ''
    df_new['mgf_Charge'] = ''
    df_new['Delt_PM'] = ''
    df_new['Delt_RT'] = ''
    df_new['Delt_Charge'] = ''
    df_new['mgf_scan#'] = ''
    df_new['204_Intensity'] = ''
    df_new['186_Intensity'] = ''
    df_new['168_Intensity'] = ''
    df_new['144_Intensity'] = ''
    df_new['138_Intensity'] = ''
    df_new['126_Intensity'] = ''

    
    
    
    #finding BEST psm match
    df_new_hcd = df_new[df_new['Fragment\r\nType'] == 'hcd']


    i = 0
    j = 0


    #scans through byos outputs (i is each PSM row found by byos)
    for i in range(len(df_new_hcd)):

        Delt_PM = 1
        Delt_RT = 1

        #scans through extracted mgf MS2 event matrix
        for j in range(len(test)):



            #retention time
            if ( abs(df_new_hcd.iloc[i,26] - test.iloc[j,9]) <=  Delt_RT):


                #precursor mass        
                if ( abs(df_new_hcd.iloc[i,15] - test.iloc[j,10]) <= Delt_PM ):


                    #charge
                    if (df_new_hcd.iloc[i,14] == test.iloc[j,11]):

                        #Set 134/144 ratio from mgf file to byos report
                        df_new_hcd.iloc[i,41] = test.iloc[j,15]

                        #add mgf PM
                        df_new_hcd.iloc[i,42] = test.iloc[j,10]

                        #add mgf RT
                        df_new_hcd.iloc[i,43] = test.iloc[j,9]

                        #add mgf charge
                        df_new_hcd.iloc[i,44] = test.iloc[j,11]


                        #Update Delts
                        Delt_PM = abs(df_new_hcd.iloc[i,15] - test.iloc[j,10])
                        Delt_RT = abs(df_new_hcd.iloc[i,26] - test.iloc[j,9])


                        df_new_hcd.iloc[i,45] = Delt_PM
                        df_new_hcd.iloc[i,46] = Delt_RT
                        df_new_hcd.iloc[i,47] = df_new_hcd.iloc[i,14] - test.iloc[j,11]

                        #adding mgf scan #
                        df_new_hcd.iloc[i,48] = test.iloc[j,8]

                        #adding glyco ion intensity
                        df_new_hcd.iloc[i,49] = test.iloc[j,1]
                        df_new_hcd.iloc[i,50] = test.iloc[j,2]
                        df_new_hcd.iloc[i,51] = test.iloc[j,3]
                        df_new_hcd.iloc[i,52] = test.iloc[j,4]
                        df_new_hcd.iloc[i,53] = test.iloc[j,5]
                        df_new_hcd.iloc[i,54] = test.iloc[j,6]
                        


            j = j + 1


        i = i + 1

    return df_new_hcd



In [12]:
#Example


#read input dataframes (mgf file & byos PSM report)

#Add file paths for YOUR_FILE_PATH

f03 = pd.read_csv(r'/YOUR_FILE_PATH/BD20210216_RB_PSD_GlcNac_Fxn_03.mgf', sep = ',', 
                             na_values = '')

f03_byos = pd.read_csv(r'/YOUR_FILE_PATH/byos_fxn03.csv', sep = ',', 
                            na_values = '')

#Interpret mgf file
f03 = SpecNAc(f03)

#Match PSMs in byos PSM report to mgf MS2 events
m03 = MgfMatch(f03_byos, f03)


#Export updated byos PSM report
m03.to_csv(r'/YOUR_FILE_PATH/fxn03_matchedmgf.csv', index = False)